In [3]:
#guava :

import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import re

# Updated dataset paths for guava
train_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_guava_ext/train'
test_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_guava_ext/test'
validation_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_guava_ext/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define checkpoint path for guava
checkpoint_dir = "./checkpoints_guava"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "model_checkpoint_guava_epoch_{epoch:02d}.h5")

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze all base model layers initially
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)

# Find latest checkpoint for guava
def get_latest_checkpoint():
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith("model_checkpoint_guava_epoch")]
    if not checkpoint_files:
        return None, 0

    # Extract epoch numbers
    epochs_trained = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in checkpoint_files]
    latest_epoch = max(epochs_trained)
    latest_checkpoint = os.path.join(checkpoint_dir, f"model_checkpoint_guava_epoch_{latest_epoch:02d}.h5")
    return latest_checkpoint, latest_epoch

latest_checkpoint, last_trained_epoch = get_latest_checkpoint()

# Load the latest model checkpoint if available
if latest_checkpoint and os.path.exists(latest_checkpoint):
    print(f"Resuming training from checkpoint: {latest_checkpoint}")
    model = load_model(latest_checkpoint)
else:
    print("No saved model found. Starting fresh.")

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ModelCheckpoint callback for guava
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

# Training Phases
epochs_phase_1 = 10
epochs_phase_2 = 5
epochs_phase_3 = 5

# Determine phase and continue training
if last_trained_epoch < epochs_phase_1:
    print(f"Starting Phase 1 (continuing from epoch {last_trained_epoch + 1})")
    history1 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1

# Unfreeze top 5 layers and continue training
if last_trained_epoch < epochs_phase_1 + epochs_phase_2:
    for layer in vgg19_base.layers[-5:]:
        layer.trainable = True
    for layer in densenet_base.layers[-5:]:
        layer.trainable = True

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print(f"Starting Phase 2 (continuing from epoch {last_trained_epoch + 1})")
    history2 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1 + epochs_phase_2

# Unfreeze all layers and fine-tune
if last_trained_epoch < epochs_phase_1 + epochs_phase_2 + epochs_phase_3:
    for layer in vgg19_base.layers:
        layer.trainable = True
    for layer in densenet_base.layers:
        layer.trainable = True

    model.compile(
        optimizer=Adam(learning_rate=1e-6),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print(f"Starting Phase 3 (continuing from epoch {last_trained_epoch + 1})")
    history3 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2 + epochs_phase_3,
        callbacks=[checkpoint_callback]
    )

# Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator)
val_loss, val_accuracy = model.evaluate(validation_generator)

print(f'Test Loss (Guava): {test_loss}')
print(f'Test Accuracy (Guava): {test_accuracy}')
print(f'Validation Loss (Guava): {val_loss}')
print(f'Validation Accuracy (Guava): {val_accuracy}')

Found 2138 images belonging to 2 classes.
Found 459 images belonging to 2 classes.
Found 459 images belonging to 2 classes.
Resuming training from checkpoint: ./checkpoints_guava\model_checkpoint_guava_epoch_15.h5


Starting Phase 3 (continuing from epoch 16)
Epoch 16/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9880 - loss: 0.0385 
Epoch 16: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_16.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 989s 15s/step - accuracy: 0.9880 - loss: 0.0385 - val_accuracy: 0.9888 - val_loss: 0.0390
Epoch 17/20
 1/66 ━━━━━━━━━━━━━━━━━━━━ 12:46 12s/step - accuracy: 1.0000 - loss: 0.0145
Epoch 17: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_17.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 1.0000 - loss: 0.0145 - val_accuracy: 0.9888 - val_loss: 0.0397
Epoch 18/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9960 - loss: 0.0290 
Epoch 18: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_18.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 946s 14s/step - accuracy: 0.9959 - loss: 0.0290 - val_accuracy: 0.9888 - val_loss: 0.0380
Epoch 19/20
 1/66 ━━━━━━━━━━━━━━━━━━━━ 13:05 12s/step - accuracy: 1.0000 - loss: 0.0463
Epoch 19: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_19.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 1.0000 - loss: 0.0463 - val_accuracy: 0.9888 - val_loss: 0.0379
Epoch 20/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.9881 - loss: 0.0455 
Epoch 20: saving model to ./checkpoints_guava\model_checkpoint_guava_epoch_20.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 940s 14s/step - accuracy: 0.9882 - loss: 0.0454 - val_accuracy: 0.9888 - val_loss: 0.0375
15/15 ━━━━━━━━━━━━━━━━━━━━ 150s 10s/step - accuracy: 0.9910 - loss: 0.0406
15/15 ━━━━━━━━━━━━━━━━━━━━ 154s 10s/step - accuracy: 0.9848 - loss: 0.0384
Test Loss (Guava): 0.04588472470641136
Test Accuracy (Guava): 0.9869281053543091
Validation Loss (Guava): 0.03684084117412567
Validation Accuracy (Guava): 0.9891067743301392
